In [1]:
import os
os.chdir('/workspaces/work_utils2')

In [2]:
import pandas as pd
import random_dataframe as rd
import query_tools as qt

## Generate Sample Data

In [3]:
import json
with open('/workspaces/work_utils2/notebooks/creating_random_data/example_03.json', 'r') as f:
    specs = json.load(f)
df = rd.create_dataframe(specs, n_rows=1000, random_seed=42)
df.sample(5)

,id,name,open_date,open_week,open_month,customer_id,score,active,value,category,product_category,quantity,price,is_discounted
312,1313,AjiGFVfO6J67bCdaszy,2024-05-14,2024-03-01,2024-03-31,348,80.772624,True,74.768991,Low,Electronics,2,40.020129,False
75,1076,None,2024-12-18,2024-02-09,2024-08-31,624,74.013916,True,120.817690,Medium,Clothing,2,31.301804,False
2,1003,Gl5v8RyWA6PB7po99U9YR2Z4c,2024-09-14,2024-07-26,2024-06-30,56,74.505334,False,122.809963,Low,Clothing,3,47.137104,False
444,1445,None,2024-11-25,2024-01-26,2024-07-31,813,91.764234,False,118.048192,Medium,Electronics,3,10.000000,False
828,1829,joPC4w6q7iXJipJVYG2,2024-10-13,2024-12-13,2024-01-31,591,65.343321,True,116.235681,Low,Electronics,3,31.272944,False


# Load Filters

In [4]:
with open('/workspaces/work_utils2/notebooks/query_data/example_02.json', 'r') as f:
    named_filters = json.load(f)
    
# Display available filter names
print("Available filters:")
for filter_name in named_filters.keys():
    print(f"- {filter_name}")

Available filters:
- active_customers
- high_value_items
- electronics_products
- q1_orders
- discounted_items
- high_quantity
- premium_products


In [5]:
mask = qt.filters.apply_filter(df, named_filters["active_customers"])
filtered_df = df[mask]
filtered_df.sample(5)

,id,name,open_date,open_week,open_month,customer_id,score,active,value,category,product_category,quantity,price,is_discounted
591,1592,ZVTn6G,2024-01-02,2024-11-08,2024-04-30,457,88.351127,True,98.172316,Low,Electronics,3,48.689850,False
194,1195,None,2024-11-15,2024-11-01,2024-06-30,963,66.819978,True,95.768844,Low,Electronics,5,40.099200,True
839,1840,74fahGYQuxyU24QsC0j3lG3gq88,2024-08-27,2024-07-26,2024-12-31,990,76.552461,True,102.072213,High,Electronics,2,25.087702,False
321,1322,2cnLX,2024-04-30,2024-08-16,2024-03-31,800,70.090511,True,96.289736,Medium,Books,1,48.094571,False
541,1542,mMe5jjLA7Yuha,2024-03-13,2024-08-09,2024-03-31,650,71.702017,True,113.304289,Low,Electronics,2,10.000000,False


## Example 1: Basic groupby and aggregation

In [6]:
agg_config = {
    "type": "aggregation",
    "spec": {
        "groupby": ["product_category", "category"],
        "aggregations": [
            {"column": "customer_id", "function": "nunique", "name": "Customer Count"},
            {"column": "price", "function": "mean", "name": "Avg Price"},
            {"column": "quantity", "function": "sum", "name": "Total Quantity"},
            {"column": "is_discounted", "function": "mean", "name": "Discount Rate"},
        ]
    }
}
qt.aggregations.process_aggregation(filtered_df, agg_config)

,product_category,category,Customer Count,Avg Price,Total Quantity,Discount Rate
0,Books,High,6,33.634237,18,0.500000
1,Books,Low,71,30.688039,196,0.310811
2,Books,Medium,29,33.476515,74,0.200000
3,Clothing,High,20,37.380248,46,0.300000
4,Clothing,Low,118,35.022643,360,0.300813
5,Clothing,Medium,59,34.645147,162,0.312500
6,Electronics,High,25,32.382467,67,0.400000
7,Electronics,Low,132,28.271796,384,0.357143
8,Electronics,Medium,73,34.171239,225,0.263158
9,Food,High,13,40.733715,23,0.384615


## Example 2: Binning a numeric column with equal width bins

In [7]:
agg_config = {
    "type": "aggregation",
    "spec": {
        "groupby": [
            {"column": "price", "bins": {"method": "equal_width", "n_bins": 5, "name": "Price Bins"}},
        ],
        "aggregations": [
            {"column": "value", "function": "sum", "name": "Total Value"},
            {"column": "quantity", "function": "sum", "name": "Total Quantity"}
        ]
    }
}
qt.aggregations.process_aggregation(df, agg_config)

,Price Bins,Total Value,Total Quantity
0,167 to 245,880.166161,23
1,245 to 324,194.475197,4
2,324 or More,358.009203,8
3,88 to 167,4829.158104,117
4,Less Than 88,94196.520722,2534


In [8]:
agg_config = {
    "type": "aggregation",
    "spec": {
        "groupby": [
            {"column": "price", "bins": {"method": "equal_width", "n_bins": 5, "name": "Price Bins", "sortable": True}},
        ],
        "aggregations": [
            {"column": "value", "function": "sum", "name": "Total Value"},
            {"column": "quantity", "function": "sum", "name": "Total Quantity"}
        ]
    }
}
qt.aggregations.process_aggregation(df, agg_config)

,Price Bins,Total Value,Total Quantity
0,1: Less Than 88,94196.520722,2534
1,2: 88 to 167,4829.158104,117
2,3: 167 to 245,880.166161,23
3,4: 245 to 324,194.475197,4
4,5: 324 or More,358.009203,8


## Example 3: Binning with custom edges and labels

In [9]:
agg_config = {
    "type": "aggregation",
    "spec": {
        "groupby": [
            {"column": "price", "bins": {
                "method": "custom", 
                "bin_edges": [18, 25, 35, 45, 55, 65],
                "bin_labels": ["00-18", "18-24", "25-34", "35-44", "45-54", "55-64", "65+"],
                "name": "Price Bins",
            }}
        ],
        "aggregations": [
            {"column": "value", "function": "sum", "name": "Total Value"},
            {"column": "customer_id", "function": "nunique", "name": "Customer Count"}
        ]
    }
}
qt.aggregations.process_aggregation(df, agg_config)


,Price Bins,Total Value,Customer Count
0,00-18,46966.832349,364
1,18-24,12185.637034,116
2,25-34,11707.675429,110
3,35-44,7137.293385,69
4,45-54,6867.745599,65
5,55-64,3693.727823,37
6,65+,11899.417770,110


In [10]:
pivot_config = {
    "type": "pivot",
    "spec": {
        "index": "product_category",
        "columns": "category",
        "values": "value",
        "aggfunc": "sum",
        "fill_value": 0,
        "margins": True,
        "margins_name": "Total"
    }
}
qt.aggregations.process_aggregation(df, pivot_config)

category,product_category,High,Low,Medium,Total
0,Books,1165.628503,9560.044247,3736.690296,14462.363047
1,Clothing,2433.658772,15157.962558,7739.381104,25331.002434
2,Electronics,2719.387961,16267.357758,9490.468233,28477.213952
3,Food,1442.401962,12467.007792,6378.820131,20288.229885
4,Home,1231.971498,6789.090638,3878.457935,11899.520070
5,Total,8993.048697,60241.462993,31223.817698,100458.329388


## Example 5: Summary statistics by group

In [14]:
summary_config = {
    "type": "summary",
    "spec": {
        "groupby": "product_category",
        "columns": ["price", "quantity", "value"],
        "statistics": ["count", "mean", "range", "geo_mean"]
    }
}
qt.aggregations.process_aggregation(df, summary_config)

,product_category,price_count,price_mean,price_range,price_geo_mean,quantity_count,quantity_mean,quantity_range,quantity_geo_mean,value_count,value_mean,value_range,value_geo_mean
0,Books,143,31.386237,176.448383,22.795974,143,2.692308,8,2.302882,143,101.135406,82.853026,99.976612
1,Clothing,254,34.698821,392.300086,23.366865,254,2.763780,9,2.352013,254,99.728356,90.413164,98.420192
2,Electronics,284,31.499103,257.572275,22.235989,284,2.771127,9,2.317143,284,100.271880,74.399552,99.216818
3,Food,200,34.697477,327.646756,23.512490,200,2.480000,9,2.080173,200,101.441149,78.449167,100.450843
4,Home,119,35.230801,239.562066,24.081508,119,2.655462,8,2.242771,119,99.995967,96.890509,98.613580


## Example 7: Equal frequency binning (quantiles)


In [16]:
agg_config = {
    "type": "aggregation",
    "spec": {
        "groupby": [
            {"column": "value", "bins": {"method": "equal_freq", "n_bins": 4, "name": "Value Bins", "sortable": True}},
        ],
        "aggregations": [
            {"column": "value", "function": "sum", "name": "Total Value"},
            {"column": "quantity", "function": "count", "name": "Transaction Count"}
        ]
    }
}
result7 = qt.aggregations.process_aggregation(df, agg_config)
print(result7)

         Value Bins   Total Value  Transaction Count
0  1:  Less Than 90  20368.567790                250
1     2:  90 to 100  23905.304993                250
2    3:  100 to 110  26347.441763                250
3   4:  110 or More  29837.014841                250
